# Lecture 1106: More Altair charts

In [1]:
import pandas as pd
import altair as alt

## Import data

In [2]:
# Note the file name below: I use ../ to go up one directory
berkeley_311 = pd.read_csv('../1023/berkeley_311_clean.csv', 
    dtype={
        'Case_ID': object,
    },
    parse_dates=['Date_Opened', 'Date_Closed']
)
berkeley_311

Case_ID         Date_Opened Case_Status         Date_Closed  \
0       121000877593 2021-09-16 06:23:23      Closed 2021-09-20 11:22:22   
1       121000876647 2021-09-13 10:50:00        Open                 NaT   
2       121000809740 2020-11-06 16:51:00      Closed 2020-11-09 01:52:57   
3       121000809739 2020-11-06 16:38:00      Closed 2020-11-09 01:41:12   
4       121000793663 2020-09-01 11:32:00      Closed 2020-09-01 11:36:00   
...              ...                 ...         ...                 ...   
704668  121001011148 2023-06-12 13:06:00      Closed 2023-07-06 13:34:09   
704669  121001014556 2023-06-29 14:01:00      Closed 2023-07-06 15:26:38   
704670  121001005559 2023-05-17 11:56:00      Closed 2023-07-06 12:07:40   
704671  121000883511 2021-10-13 10:59:00      Closed 2023-07-06 06:55:14   
704672  121001013680 2023-06-26 12:42:00      Closed 2023-07-06 11:47:40   

                                    Request_Category  \
0       Facilities, Electrical & Property Management   
1                               Refuse and Recycling   
2                      General Questions/information   
3                      General Questions/information   
4                 Other Account Services and Billing   
...                                              ...   
704668                          Refuse and Recycling   
704669                          Refuse and Recycling   
704670                          Refuse and Recycling   
704671                   Parks, Trees and Vegetation   
704672                          Refuse and Recycling   

                   Request_SubCategory                  Request_Detail  \
0       Parks/Marina Building Services                    Keys / Locks   
1                          Residential        Residential Bulky Pickup   
2                        Miscellaneous   Miscellaneous Service Request   
3                        Miscellaneous   Miscellaneous Service Request   
4                               Marina     Payment Collection - Marina   
...                                ...                             ...   
704668                      Commercial      Commercial Site Inspection   
704669                         Request                     Cart Repair   
704670                     Residential     Residential Site Inspection   
704671                           Trees                    Tree Pruning   
704672                     Residential  Residential Cart Size Decrease   

       Object_Type            APN  \
0         Property            NaN   
1         Property  054 180702800   
2       Individual            NaN   
3         Property  060 249305600   
4       Individual            NaN   
...            ...            ...   
704668    Property  058 217100500   
704669    Property  053 167101600   
704670    Property  055 191404100   
704671    Property  060 244500103   
704672    Property  052 156509000   

                                           Street_Address      City State  \
0       Intersection of Browning and Addison, BERKELEY...  Berkeley    CA   
1                                         1722 DWIGHT WAY  Berkeley    CA   
2                                                     NaN  Berkeley    CA   
3                                  1411 GRIZZLY PEAK BLVD  Berkeley    CA   
4                                                     NaN  Berkeley    CA   
...                                                   ...       ...   ...   
704668                                      1924 CEDAR ST  Berkeley    CA   
704669                                     1513 OREGON ST  Berkeley    CA   
704670                                 2326 SPAULDING AVE  Berkeley    CA   
704671                               1700 HOPKINS ST PARK  Berkeley    CA   
704672                                   2946 MAGNOLIA ST  Berkeley    CA   

       Neighborhood   Latitude   Longitude                      Location  \
0          Berkeley        NaN         NaN                           NaN   
1          Berke

In [3]:
berkeley_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704673 entries, 0 to 704672
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Case_ID              704673 non-null  object        
 1   Date_Opened          704673 non-null  datetime64[ns]
 2   Case_Status          704673 non-null  object        
 3   Date_Closed          667173 non-null  datetime64[ns]
 4   Request_Category     704673 non-null  object        
 5   Request_SubCategory  704673 non-null  object        
 6   Request_Detail       704673 non-null  object        
 7   Object_Type          704673 non-null  object        
 8   APN                  413081 non-null  object        
 9   Street_Address       455047 non-null  object        
 10  City                 704673 non-null  object        
 11  State                704673 non-null  object        
 12  Neighborhood         704673 non-null  object        
 13  Latitude      

## Text and dimensions
- Add a title and subtitle
- Specify height and width
- Customize axes

In [4]:
calls_by_month = berkeley_311.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='M')]).count()[['Case_ID']].reset_index()

# Rename column to `COUNT`
calls_by_month.rename(columns={'Case_ID': 'Count'}, inplace=True)

In [9]:
alt.Chart(calls_by_month).mark_bar().encode(
    x='Date_Opened:T',
    y='Count'
)

alt.Chart(...)

In [26]:
alt.Chart(calls_by_month).mark_bar().encode(
    x=alt.X('Date_Opened:T', axis=alt.Axis(tickCount=10, title='')),
    # y='Count' # comment this line out, then uncomment the below line
    y=alt.Y('Count', axis=alt.Axis(tickCount=7, title=''))
).properties(
    title={
        'text': 'Berkeley 311 calls',
        'subtitle' : ['Number of cases per month']
    },
    height=200,
    width=600
)

alt.Chart(...)

How would you customize the x-axis?

## Line chart

Line charts are good for rates, or if you have multiple series.

In [27]:
berkeley_311['Request_Category'].value_counts()

Request_Category
Refuse and Recycling                            314355
General Questions/information                   167258
Streets, Utilities, and Transportation           76552
Other Account Services and Billing               68893
Parks, Trees and Vegetation                      26199
Traffic and Transportation                       16668
Facilities, Electrical & Property Management      9797
Public Records Act                                7992
Government Activity                               5643
Graffiti and Vandalism                            4789
Business License                                  3265
Environmental Services and Programs               2738
Equipment Maintenance                              491
Disability Compliance                               32
Outside Agencies                                     1
Name: count, dtype: int64

In [28]:
categories_by_month = berkeley_311.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='M'), 'Request_Category']).count()[['Case_ID']].reset_index()

In [29]:
# rename cols
categories_by_month.columns = ['Month_Opened', 'Request_Category', 'Cases'] # we've been using `Count` but I switched to Cases here for no reason.

In [30]:
categories_by_month

Month_Opened                              Request_Category  Cases
0      2010-02-28  Facilities, Electrical & Property Management      3
1      2010-02-28                          Refuse and Recycling      1
2      2010-03-31           Environmental Services and Programs      9
3      2010-03-31  Facilities, Electrical & Property Management      8
4      2010-03-31                 General Questions/information    106
...           ...                                           ...    ...
1716   2023-10-31            Other Account Services and Billing     48
1717   2023-10-31                   Parks, Trees and Vegetation    121
1718   2023-10-31                          Refuse and Recycling   1266
1719   2023-10-31        Streets, Utilities, and Transportation    451
1720   2023-10-31                    Traffic and Transportation    151

[1721 rows x 3 columns]

In [31]:
# How to get percentage of total by month
categories_by_month['Case_Rate'] = categories_by_month['Cases'] / categories_by_month.groupby('Month_Opened')['Cases'].transform('sum')
categories_by_month

Month_Opened                              Request_Category  Cases  \
0      2010-02-28  Facilities, Electrical & Property Management      3   
1      2010-02-28                          Refuse and Recycling      1   
2      2010-03-31           Environmental Services and Programs      9   
3      2010-03-31  Facilities, Electrical & Property Management      8   
4      2010-03-31                 General Questions/information    106   
...           ...                                           ...    ...   
1716   2023-10-31            Other Account Services and Billing     48   
1717   2023-10-31                   Parks, Trees and Vegetation    121   
1718   2023-10-31                          Refuse and Recycling   1266   
1719   2023-10-31        Streets, Utilities, and Transportation    451   
1720   2023-10-31                    Traffic and Transportation    151   

      Case_Rate  
0      0.750000  
1      0.250000  
2      0.041475  
3      0.036866  
4      0.488479  
...         ...  
1716   0.016327  
1717   0.041156  
1718   0.430612  
1719   0.153401  
1720   0.051361  

[1721 rows x 4 columns]

In [32]:
refuse = categories_by_month[categories_by_month['Request_Category'] == 'Refuse and Recycling'].copy()
refuse

Month_Opened      Request_Category  Cases  Case_Rate
1      2010-02-28  Refuse and Recycling      1   0.250000
6      2010-03-31  Refuse and Recycling     33   0.152074
13     2010-04-30  Refuse and Recycling    780   0.374280
21     2010-05-31  Refuse and Recycling    710   0.338902
28     2010-06-30  Refuse and Recycling   1154   0.419789
...           ...                   ...    ...        ...
1677   2023-06-30  Refuse and Recycling   2008   0.408130
1687   2023-07-31  Refuse and Recycling   1708   0.395370
1698   2023-08-31  Refuse and Recycling   2355   0.444256
1708   2023-09-30  Refuse and Recycling   1914   0.430596
1718   2023-10-31  Refuse and Recycling   1266   0.430612

[165 rows x 4 columns]

In [41]:
alt.Chart(refuse).mark_line().encode(
    x=alt.X('Month_Opened'),
    y=alt.Y('Case_Rate', axis=alt.Axis(format='%', title='')),
).properties(
    title={
        'text': 'Berkeley 311 calls',
        'subtitle' : ['Rate of Refuse and Recycling cases per month']
    }
)

alt.Chart(...)

## Check out Altair's examples

Altair has a [gallery of examples](https://altair-viz.github.io/gallery/index.html). Let's look at them together.

In [42]:
categories_by_month

Month_Opened                              Request_Category  Cases  \
0      2010-02-28  Facilities, Electrical & Property Management      3   
1      2010-02-28                          Refuse and Recycling      1   
2      2010-03-31           Environmental Services and Programs      9   
3      2010-03-31  Facilities, Electrical & Property Management      8   
4      2010-03-31                 General Questions/information    106   
...           ...                                           ...    ...   
1716   2023-10-31            Other Account Services and Billing     48   
1717   2023-10-31                   Parks, Trees and Vegetation    121   
1718   2023-10-31                          Refuse and Recycling   1266   
1719   2023-10-31        Streets, Utilities, and Transportation    451   
1720   2023-10-31                    Traffic and Transportation    151   

      Case_Rate  
0      0.750000  
1      0.250000  
2      0.041475  
3      0.036866  
4      0.488479  
...         ...  
1716   0.016327  
1717   0.041156  
1718   0.430612  
1719   0.153401  
1720   0.051361  

[1721 rows x 4 columns]

In [49]:
categories_by_month['Request_Category'].value_counts()

Request_Category
Facilities, Electrical & Property Management    165
Refuse and Recycling                            165
General Questions/information                   164
Other Account Services and Billing              164
Streets, Utilities, and Transportation          164
Traffic and Transportation                      164
Equipment Maintenance                           126
Public Records Act                              119
Graffiti and Vandalism                          119
Government Activity                             115
Parks, Trees and Vegetation                     112
Environmental Services and Programs              90
Business License                                 32
Disability Compliance                            21
Outside Agencies                                  1
Name: count, dtype: int64

In [50]:
categories = [
    'Facilities, Electrical & Property Management',
    'Refuse and Recycling',
    'General Questions/information',
    'Other Account Services and Billing',
    'Streets, Utilities, and Transportation'
]

top_5 = categories_by_month[
    categories_by_month['Request_Category'].isin(categories)
].copy()

top_5

Month_Opened                              Request_Category  Cases  \
0      2010-02-28  Facilities, Electrical & Property Management      3   
1      2010-02-28                          Refuse and Recycling      1   
3      2010-03-31  Facilities, Electrical & Property Management      8   
4      2010-03-31                 General Questions/information    106   
5      2010-03-31            Other Account Services and Billing     34   
...           ...                                           ...    ...   
1712   2023-10-31  Facilities, Electrical & Property Management     39   
1713   2023-10-31                 General Questions/information    772   
1716   2023-10-31            Other Account Services and Billing     48   
1718   2023-10-31                          Refuse and Recycling   1266   
1719   2023-10-31        Streets, Utilities, and Transportation    451   

      Case_Rate  
0      0.750000  
1      0.250000  
3      0.036866  
4      0.488479  
5      0.156682  
...         ...  
1712   0.013265  
1713   0.262585  
1716   0.016327  
1718   0.430612  
1719   0.153401  

[822 rows x 4 columns]

In [61]:
alt.Chart(top_5).mark_line().encode(
    column='Request_Category:N',
    # row=alt.Row("Request_Category:N"),
    x=alt.X('Month_Opened:T', axis=alt.Axis(title='')),
    y=alt.Y('Case_Rate', axis=alt.Axis(format='%', title='')),
    # color='Request_Category:N'
).properties(
    title={
        'text': 'Berkeley 311 calls',
        'subtitle' : ['Rate of cases per month']
    },
    height=100
)

alt.Chart(...)